In [ ]:
%matplotlib notebook
import concurrent.futures
import importlib
import os
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from skimage.io import imread, imsave
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import keras.backend as K
from skimage.color import label2rgb
from keras import Model
from keras.optimizers import Nadam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical
from pycocotools.coco import COCO
from collections import Counter
from sklearn.utils import compute_class_weight

from abyss_deep_learning.keras.detection import MaskRcnnDataset
from abyss_deep_learning.utils import ann_to_mask
from mrcnn.model import MaskRCNN, data_generator
from mrcnn.utils import Dataset as DatasetBase


from skimage.morphology import remove_small_holes

In [ ]:
def load_config(path, num_classes):
    assert num_classes or dataset_train
    spec = importlib.util.spec_from_file_location(
        "maskrcnn_config", path)
    config_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(config_module)
    config = config_module.Config()
    config.NUM_CLASSES = num_classes
    input_shape = config.IMAGE_SHAPE
    return config

# Setup Variables

In [ ]:
image_dir = None
categories = None
num_classes = 2
use_balanced_set = False
use_class_weights = True
config_file = "/home/docker/src/abyss/deep-learning/configs/MaskRCNN_default_config.py"

# Setup Data

In [ ]:
database_dir = "/data/acfr/collated/2017-summer-lettuce"
dataset_name = "20170329T000000"
dataset_files = {
    'train': os.path.join(database_dir, "{:s}/train.json".format(dataset_name)),
    'val': os.path.join(database_dir, "{:s}/val.json".format(dataset_name)),
    'test': os.path.join(database_dir, "{:s}/test.json".format(dataset_name))
}
dataset = {
    'names': list(dataset_files.keys()),
    'classes': [], # MUST FILL IN
    'class_weights': {name: None for name in dataset_files.keys()},
    'ids' : {},
    'gens': {},
    'data': {},
    'coco': {},
    'config': load_config(config_file, num_classes=num_classes)
}
dataset['config'].NUM_CLASSES = num_classes
dataset['name'] = dataset_name.replace("/", "-")
dataset['config'].NAME = dataset['name']


In [ ]:
num_classes = len(MaskRcnnDataset(dataset_files['train']).coco.cats) + 1

for name, path in dataset_files.items():
    coco = MaskRcnnDataset(path)
    
    ids = coco.image_ids
    gen = data_generator(coco, dataset['config'], shuffle=True, augmentation=None, detection_targets=False)
    print("{:s}: {:d} images".format(name, len(ids)))
    dataset['coco'][name] = coco
    dataset['ids'][name] = ids
    dataset['gens'][name] = gen
    
dataset['config'].STEPS_PER_EPOCH = len(dataset['ids']['train']) // dataset['config'].BATCH_SIZE
dataset['classes'] = sorted([cat['id'] for cat in dataset['coco']['train'].coco.cats.values()])
dataset['config'].display()

In [ ]:
%%timeit -n5 -r1
image, target = dataset['coco']['train'].sample()
print(image.shape, target[0].shape, target[1])


In [ ]:
for inputs, targets in dataset['gens']['train']:
    images, image_meta, rpn_match, rpn_bbox, gt_class_ids, gt_boxes, gt_masks = inputs
    print("images.shape", images.shape)
    print("image_meta.shape", image_meta.shape)
    print("rpn_match.shape", rpn_match.shape)
    print("rpn_bbox.shape", rpn_bbox.shape)
    print("gt_class_ids.shape", gt_class_ids.shape)
    print("gt_boxes.shape", gt_boxes.shape)
    print("gt_masks.shape", gt_masks.shape)
    print("images min/max", np.min(images), np.max(images))
    break
    
plt.figure()
num_rows = 3
print("Left to right: ground truth samples from ", end='')
for j in range(num_rows):
    for i, name in enumerate(dataset['names']):
        plt.subplot(num_rows, 3, 3 * j + i + 1)
    #     print(data[0].shape, data[1], (np.min(data[0]), np.max(data[0])))
        image, targets = dataset['coco'][name].sample()
        print(image.dtype, image.shape)
        plt.imshow((image))
#         plt.title(', '.join([caption_map_r[int(cap_id)] for cap_id in np.argwhere(label)]))
        print(name, end=', ')
        plt.axis('off')


# def display_instances(images, masks, boxes, class_ids):
#     valid = np.all(boxes, axis=1)
#     class_ids = class_ids[valid]
#     masks = masks[..., valid]#.transpose((2, 0, 1))
#     boxes = boxes[valid, ...]
#     print("masks.shape", masks.shape)
# #     for i, class_id in enumerate(class_ids):
# #         masks[..., i] *= class_id
#     print(boxes.shape, masks.shape, images.shape, class_ids.shape)
    
#     labels = expand_mask(boxes, masks, images.shape).astype(np.uint8)
#     labels = np.any(labels, axis=2)
# #         labels *= np.arange(1, labels.shape[-1] + 1).astype(np.uint8)
# #         print(labels)
#     labels = resize(labels, images.shape[0:2], order=0)
#     return label2rgb(labels, images.astype(np.uint8), bg_label=0)

# def display_instances_gen(gen, add_mean=None):
#     ''''batching == 1 support ONLY'''
#     for ((images, image_meta, rpn_match, rpn_bbox, gt_class_ids, gt_boxes, gt_masks), targets) in gen:
#         image = images[0] + add_mean if add_mean is not None else images[0]
#         yield (display_instances(image, gt_masks[0], gt_boxes[0], gt_class_ids[0]),)

# displays = [
#     display[0] for display in 
#         head_gen(
#             display_instances_gen(
#                 data_generator(dataset_train, config)
#             , add_mean=config.MEAN_PIXEL),
#         first=4)]
# plt.figure()
# vis_square(np.array(displays))
# plt.tight_layout()

In [ ]:
if use_class_weights:
    for name, ds in dataset['coco'].items():
        print("{:s} {:s} class stats {:s}".format('=' * 8, name, '=' * 8))
        y = [ann['category_id'] for ann in ds.coco.anns.values() if 'segmentation' in ann]
        count = np.array(list(dict(sorted(Counter(y).items(), key=lambda x: x[0])).values()))
        spread = {i: float(v.round(2)) for i, v in enumerate(count / np.sum(count))}
        class_weights = compute_class_weight('balanced', dataset['classes'], y)
        class_weights = {i: float(np.round(v, 3)) for i, v in enumerate(class_weights)}
        dataset['class_weights'][name] = class_weights
        a = np.array(list(dataset['class_weights'][name].values()))
        
        print("class weights:".format(name))
        print(" ", class_weights)
        print("class cover fractions:\n  ", spread )


In [ ]:
class Experiment(object):
    def __init__(self, config, model_dir):
        self.epoch = 0
        self.model = None
        self.config = config
        self.model_dir = model_dir
        self.compiled = False
    
    def create(self, model_path=None, train=False, fresh_heads=False):
        if not model_path:
            model_path = '/data/models/mask_rcnn_coco.h5'
            
        if not train:
            self.config.IMAGES_PER_GPU = 1
            self.config.BATCH_SIZE = 1
        self.model = None
        K.clear_session()
        
        self.model = MaskRCNN(
            mode=("training" if train else "inference"),
            config=self.config, model_dir=self.model_dir)
        if model_path: 
            exclude = [
                "mrcnn_class_logits", "mrcnn_bbox_fc",
                "mrcnn_bbox", "mrcnn_mask"] if fresh_heads else []
            self.model.load_weights(model_path, by_name=True, exclude=exclude)
    
    def train(self, learning_rate, epochs, layers, **kwargs):
        return self.model.train(
            dataset['coco']['train'], dataset['coco']['val'], 
            learning_rate, epochs, layers,
            **kwargs
        )


# Train MRCNN heads

In [ ]:
model_path = None # None for COCO pretrained weights
logdir = os.path.join("/data/log/maskrcnn/{:s}".format(dataset['name']))
!mkdir -p "$logdir/models"
best_path = os.path.join(logdir, "models/best.{epoch:03d}-{val_loss:.4f}.h5")


In [ ]:
exp = Experiment(dataset['config'], logdir)
model = exp.create(model_path=model_path, train=True, fresh_heads=True)
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=5, verbose=1),
    EarlyStopping(
        monitor='val_loss', min_delta=0.0, patience=10, verbose=1, mode='auto')
]

exp.train(
    1e-3, 100, 'heads',
    augmentation=None,
    custom_callbacks=callbacks,
    no_augmentation_sources=None)

In [ ]:
saved_model_path = 'FILL_THIS_IN/mask_rcnn_20170329t000000_0040.h5'

exp = Experiment(dataset['config'], logdir)
model = exp.create(model_path=saved_model_path, train=True, fresh_heads=True)
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, cooldown=10, verbose=1),
    EarlyStopping(
        monitor='val_loss', min_delta=0.0, patience=20, verbose=1, mode='auto')
]

exp.train(
    1e-4, 200, '3+',
    augmentation=None,
    custom_callbacks=callbacks,
    no_augmentation_sources=None)
exp.model.keras_model.save_weights(os.path.join(logdir, 'final.h5'))

In [ ]:
del exp
exp = Experiment(dataset['config'], logdir)
model = exp.create(model_path=os.path.join(logdir, 'final.h5'), train=False, fresh_heads=False)

# TODO BELOW UNFINISHED

In [ ]:
from mrcnn.utils import expand_mask
from mrcnn.visualize import display_images, display_instances
from abyss_deep_learning.keras.segmentation import jaccard_index

def plot_test(gen, model, num_images=1, show=False):
    from scipy.optimize import linear_sum_assignment
    ious_list = []
    i = 0
    for ((images, image_meta, rpn_match, rpn_bbox, gt_class_ids, gt_boxes, gt_masks), targets) in gen:
        image = images[0]
        valid = np.all(gt_boxes[0], axis=1)
        class_ids = gt_class_ids[0, valid]
        masks = gt_masks[0, ..., valid].transpose((1, 2, 0))
        boxes = gt_boxes[0, valid, ...]
        
        labels = expand_mask(boxes, masks, image.shape).astype(np.uint8)
        r = model.detect([image], verbose=True)[0]
        num_pred = len(r['class_ids'])
        num_gt = len(class_ids)
        print("GTs = {:d}, Pred = {:d}".format(num_gt, num_pred))
        
        ious = np.array([[
            jaccard_index(r['masks'][..., i] , labels[..., j]) 
                for j in range(labels.shape[-1])] 
                for i in range(r['masks'].shape[-1])])
        pred_idx, gt_idx = linear_sum_assignment(1-ious)
        r['ious'] = np.array([ious[pred_idx[i], gt_idx[i]] 
                              if (i in pred_idx and i in gt_idx) else 0.0 for i in range(num_pred)])
        print("IoUs", r['ious'])
        print("Scores", r['scores'])
        ious_list.append(ious)
        class_names = ['BG'] + [cat['name'] for cat in dataset['coco']['train'].coco.cats.values()]
        if show:
            plt.figure()
            ax = plt.subplot(1, 2, 1)
            display_instances(
                image + dataset['config'].MEAN_PIXEL,
                boxes,
                masks,
                class_ids,
                class_names, ax=ax)
#             display_predictions(
#                 image + config.MEAN_PIXEL, boxes, labels, class_ids, dataset_train.class_names, ax=ax)
#             ax = plt.subplot(1, 2, 2, sharex=ax, sharey=ax)
#             display_predictions(
#                 image + config.MEAN_PIXEL, r['rois'], r['masks'], r['class_ids'],
#                 dataset_train.class_names,  r['scores'], ax=ax)
#             plt.tight_layout()
#         imsave("/tmp/maskrcnn/image.png", (image + config.MEAN_PIXEL).astype(np.uint8))
        i += 1    
        if i >= num_images:
                break
    return ious_list

ious = plot_test(dataset['gens']['test'], exp.model, num_images=1, show=True)

In [ ]:
evaluate_coco(model, dataset_val, coco_val, eval_type="segm", limit=0, image_ids=None)